### Inventory Summary

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
inventory_snapshot = spark.read.table("inventory_project.silver.wms_inventory_snapshot")
bin_lot = spark.read.table("inventory_project.silver.wms_bin_lot")


In [0]:
df_joined = inventory_snapshot.alias("inv").join(bin_lot.alias("bin"), ["bin_id", "lot_id", "serial_id"], "left")\
    .select("inv.*", "bin.lot_expiry_date", "bin.condition", "bin.received_date")
df_joined = df_joined.drop("_rescued_data", "file_path", "_ingestion_time","ingestion_time")

df_enriched = df_joined.withColumn("day_to_expiry",datediff(col("lot_expiry_date"),col("snapshot_date")))\
    .withColumn("days_in_storage",datediff(col("snapshot_date"),col("received_date")))
window_latest = Window.partitionBy("product_id", "bin_id", "lot_id").orderBy(col("snapshot_date").desc())

df_latest = (
    df_enriched
    .withColumn("rn", row_number().over(window_latest))
    .filter(col("rn") == 1)
    .drop("rn")
)
df_stock_summary = (
    df_latest.groupBy("product_id", "bin_id", "lot_id", "status", "condition", "lot_expiry_date")
    .agg(
        sum("quantity_on_hand").alias("total_quantity"),
        first("day_to_expiry").alias("days_to_expiry"),
        first("days_in_storage").alias("days_in_storage")
    )
)
df_stock_summary = (
    df_stock_summary
    .withColumn(
        "expiry_risk_flag",
        when(col("days_to_expiry") <= 30, "High Risk")
         .when(col("days_to_expiry") <= 90, "Medium Risk")
         .otherwise("Low Risk")
    )
)

In [0]:
display(df_stock_summary)

### Inbound Movement Summary

In [0]:
putaway_movements = spark.read.table("inventory_project.silver.wms_putaway_movement")
putaway_movements = putaway_movements.drop("_rescued_data", "file_path", "_ingestion_time","ingestion_time")

In [0]:
df_putaway_clean = (
    putaway_movements
    .filter(col("product_id").isNotNull() & col("to_bin").isNotNull())
    .filter(col("cdc_op") != "D")  # Ignore deleted rows for summary
)
df_putaway_summary = (
    df_putaway_clean
    .groupBy("product_id", "lot_id", "to_bin", "operator_id")
    .agg(
        sum("quantity").alias("total_moved_quantity"),
        countDistinct("movement_id").alias("total_movements"),
        max("movement_date").alias("last_movement_date")
    )
)
df_putaway_summary = (
    df_putaway_summary
    .withColumn("week", weekofyear("last_movement_date"))
    .withColumn("month", date_format("last_movement_date", "yyyy-MM"))
    .withColumn(
        "quantity_flag",
        when(col("total_moved_quantity") > 1000, "High Volume")
         .when(col("total_moved_quantity") > 500, "Medium Volume")
         .otherwise("Low Volume")
    )
)

In [0]:
display(df_putaway_summary)

### Outbound Movement Summary

In [0]:
pick_movements = spark.read.table("inventory_project.silver.wms_pick_movement")
pick_movements = pick_movements.drop("_rescued_data", "file_path", "_ingestion_time","ingestion_time")

In [0]:
df_pick_clean = (
    pick_movements
    .filter(col("product_id").isNotNull() & col("order_id").isNotNull())
    .filter(col("cdc_op") != "D")  # ignore deleted events
)
df_pick_summary = (
    df_pick_clean
    .groupBy("order_id", "product_id", "lot_id", "bin_id", "picker_id")
    .agg(
        sum("quantity").alias("total_picked_quantity"),
        countDistinct("pick_id").alias("total_picks"),
        max("pick_date").alias("last_pick_date")
    )
)
df_pick_summary = (
    df_pick_summary
    .withColumn("week", weekofyear("last_pick_date"))
    .withColumn("month", date_format("last_pick_date", "yyyy-MM"))
    .withColumn(
        "pick_volume_flag",
        when(col("total_picked_quantity") > 1000, "High Volume")
         .when(col("total_picked_quantity") > 500, "Medium Volume")
         .otherwise("Low Volume")
    )
)

In [0]:
display(df_pick_summary)

### Shipment Summary

In [0]:
df_shipment = spark.read.table("inventory_project.silver.wms_shipment")
df_shipment = df_shipment.drop("_rescued_data", "file_path", "_ingestion_time","ingestion_time")

In [0]:
df_summary = (
    df_shipment.groupBy("ship_date", "product_id", "carrier")
    .agg(
        sum(when(col("status") == "SHIPPED", col("quantity")).otherwise(0)).alias("shipped_qty"),
        sum(when(col("status") == "CANCELLED", col("quantity")).otherwise(0)).alias("cancelled_qty"),
        countDistinct(col("order_id")).alias("total_orders")
    )
    .withColumn(
        "cancellation_rate",
        when(
            (col("shipped_qty") + col("cancelled_qty")) > 0,
            round(col("cancelled_qty") / (col("shipped_qty") + col("cancelled_qty")), 3)
        ).otherwise(0)
    )
)

In [0]:
display(df_summary)

### Inventory Position Summary (Gold)

In [0]:
df_inventory_snapshot = spark.read.table("inventory_project.silver.wms_inventory_snapshot")
df_putaway_movements = spark.read.table("inventory_project.silver.wms_putaway_movement")
df_pick_movements = spark.read.table("inventory_project.silver.wms_pick_movement")

In [0]:
# --- Step 1: Aggregate snapshots (latest per bin/product/lot)
window_snap = Window.partitionBy("product_id", "bin_id", "lot_id").orderBy(col("snapshot_date").desc())

df_snap_latest = (
    df_inventory_snapshot
    .withColumn("rn", row_number().over(window_snap))
    .filter(col("rn") == 1)
    .select("product_id", "bin_id", "lot_id", col("quantity_on_hand").alias("snapshot_qty"), "snapshot_date")
)

# --- Step 2: Aggregate putaways
df_putaway_agg = (
    df_putaway_movements
    .filter(col("cdc_op").isin("I", "U"))
    .groupBy("product_id", "to_bin", "lot_id")
    .agg(
        sum("quantity").alias("putaway_qty"),
        max("movement_date").alias("last_putaway_date")
    )
    .withColumnRenamed("to_bin", "bin_id")
)

# --- Step 3: Aggregate picks
df_pick_agg = (
    df_pick_movements
    .filter(col("cdc_op").isin("I", "U"))
    .groupBy("product_id", "bin_id", "lot_id")
    .agg(
        sum("quantity").alias("pick_qty"),
        max("pick_date").alias("last_pick_date")
    )
)

# --- Step 4: Combine all
df_joined = (
    df_snap_latest
    .join(df_putaway_agg, ["product_id", "bin_id", "lot_id"], "outer")
    .join(df_pick_agg, ["product_id", "bin_id", "lot_id"], "outer")
)


# --- Step 5: Compute final stock position
df_inventory_pos = (
    df_joined
    .withColumn("snapshot_qty", coalesce(col("snapshot_qty"), lit(0)))
    .withColumn("putaway_qty", coalesce(col("putaway_qty"), lit(0)))
    .withColumn("pick_qty", coalesce(col("pick_qty"), lit(0)))
    .withColumn(
        "current_quantity",
        col("snapshot_qty") + col("putaway_qty") - col("pick_qty")
    )
    .withColumn(
        "last_movement_date",
        greatest(col("last_putaway_date"), col("last_pick_date"), col("snapshot_date"))
    )
    .select(
        "product_id", "bin_id", "lot_id",
        "snapshot_qty", "putaway_qty", "pick_qty",
        "current_quantity", "last_movement_date"
    )
)

In [0]:
bin_lot = spark.read.table("inventory_project.silver.wms_bin_lot")
final_summary = (
    df_inventory_pos.join(bin_lot, ["bin_id","lot_id"], "left")
)
final_summary = (
    final_summary
    .withColumn("isExpired", col("lot_expiry_date")< current_date())\
    .withColumn("available_for_pickup", when((col("condition").isin("Damaged","Return")) | col("isExpired"), lit(False)).otherwise(lit(True)))
    .drop("ingestion_time","file_path","_rescued_data")    
)

In [0]:
display(final_summary)

### Order Fullfilment (shipment + pick away)

In [0]:
df_pick = spark.read.table("inventory_project.silver.wms_pick_movement")
df_ship = spark.read.table("inventory_project.silver.wms_shipment")

df_pick_valid = df_pick.filter((col("cdc_op") != "D") & (col("quantity") > 0))
df_ship_valid = df_ship.filter((col("cdc_op") != "D") & (col("quantity") > 0))

df_pick_summary = (
    df_pick_valid
    .groupBy("order_id", "product_id", "lot_id")
    .agg(
        sum("quantity").alias("total_picked_qty"),
        min("pick_date").alias("first_pick_date")
    )
)

df_ship_summary = (
    df_ship_valid
    .groupBy("order_id", "product_id", "lot_id")
    .agg(
        sum("quantity").alias("total_shipped_qty"),
        max("ship_date").alias("ship_date"),
        first("carrier", ignorenulls=True).alias("carrier"),
        first("status", ignorenulls=True).alias("status")
    )
)

df_joined = (
    df_pick_summary
    .join(df_ship_summary, ["order_id", "product_id", "lot_id"], "left")
    .fillna({"total_shipped_qty": 0})
)

df_kpi = (
    df_joined
    .withColumn(
        "fulfillment_rate",
        when(col("total_picked_qty") > 0,
               col("total_shipped_qty") / col("total_picked_qty"))
        .otherwise(0)
    )
    .withColumn(
        "fulfillment_status",
        when(col("fulfillment_rate") >= 1, "Fulfilled")
        .when((col("fulfillment_rate") > 0) & (col("fulfillment_rate") < 1), "Partial")
        .otherwise("Pending")
    )
)

In [0]:
display(df_kpi)

### Warehouse KPI Gold

In [0]:
df_inventory = spark.read.table("inventory_project.silver.wms_inventory_snapshot")
df_putaway = spark.read.table("inventory_project.silver.wms_putaway_movement")
df_pick = spark.read.table("inventory_project.silver.wms_pick_movement")
df_ship = spark.read.table("inventory_project.silver.wms_shipment")
df_bin_lot = spark.read.table("inventory_project.silver.wms_bin_lot")

In [0]:
df_putaway = df_putaway.filter((col("cdc_op") != "D") & (col("quantity") > 0))
df_ship = df_ship.filter((col("cdc_op") != "D") & (col("quantity") > 0))
df_bin = df_bin_lot.filter(col("bin_id").isNotNull())
df_snapshot = df_inventory.filter(col("bin_id").isNotNull())

In [0]:
# damaged Stock percentage
df_damaged = (
    df_bin
    .groupBy()
    .agg(
        sum(when(col("condition").isin(["Damaged", "Hold"]), 1).otherwise(0)).alias("damaged_bins"),
        countDistinct("bin_id").alias("total_bins")
    )
    .withColumn("damaged_stock_pct", (col("damaged_bins") / col("total_bins")) * 100)
)
display(df_damaged)

In [0]:
# active vs inactive bins
df_bin_status = (
    df_snapshot
    .groupBy("bin_id")
    .agg(sum("quantity_on_hand").alias("total_qty"))
    .withColumn("is_active", when(col("total_qty") > 0, 1).otherwise(0))
)

df_bin_summary = (
    df_bin_status
    .agg(
        sum("is_active").alias("active_bin_count"),
        (count("bin_id") - sum("is_active")).alias("inactive_bin_count")
    )
)
display(df_bin_status)

In [0]:
df_ship_summary = (
    df_ship
    .groupBy("ship_date")
    .agg(
        sum("quantity").alias("total_shipped_qty")
    )
)
display(df_ship_summary)

In [0]:
# Average Storage Duration
df_putaway_1 = df_putaway.select("lot_id",col("movement_date").alias("recieved_date"))
df_ship_1 = df_ship.select("lot_id",col("ship_date").alias("ship_date"))

df_joined = df_putaway_1.join(df_ship_1, ["lot_id"], "inner")
df_joined = df_joined.withColumn("storage_days", datediff(col("ship_date"),col("recieved_date")))
df_joined = df_joined.groupBy("ship_date").agg(avg("storage_days").alias("avg_storage_duration"))
display(df_joined)

### Product Lifecycle Traceability

In [0]:
df_inventory = spark.read.table("inventory_project.silver.wms_inventory_snapshot")
df_putaway = spark.read.table("inventory_project.silver.wms_putaway_movement")
df_pick = spark.read.table("inventory_project.silver.wms_pick_movement")
df_ship = spark.read.table("inventory_project.silver.wms_shipment")
df_bin_lot = spark.read.table("inventory_project.silver.wms_bin_lot")

In [0]:
df_putaway = df_putaway.filter((col("cdc_op") != "D") & (col("quantity") > 0))
df_pick = df_pick.filter((col("cdc_op") != "D") & (col("quantity") > 0))
df_ship = df_ship.filter((col("cdc_op") != "D") & (col("quantity") > 0))

In [0]:
df_putaway = df_putaway.select("product_id","lot_id","to_bin","serial_id",col("movement_date").alias("recieved_date"))
df_pick = df_pick.select("product_id","lot_id","bin_id","serial_id","pick_date")
df_ship = df_ship.select("product_id","lot_id","serial_id","ship_date","status")
df_bin_lot = df_bin_lot.select("lot_id","serial_id","condition")

In [0]:
df_trace = (
    df_putaway.alias('p').
    join(df_pick.alias('pi'), ['product_id','lot_id','serial_id'], 'left').
    join(df_ship.alias('s'), ['product_id','lot_id','serial_id'], 'left').
    join(df_bin_lot.alias('b'), ['lot_id','serial_id'], 'left')
)
df_trace = df_trace.withColumn(
    "status",
    when(col("s.status").isNotNull(), col("s.status"))
     .when(col("pick_date").isNotNull(), lit("Picked"))
     .when(col("b.condition").isin(["Damaged", "hold"]), lit("Hold/Damaged"))
     .otherwise(lit("In Stock"))
)

In [0]:
df_trace_final = df_trace.select(
    "product_id", "lot_id", "serial_id", "recieved_date", "bin_id", "pick_date", "ship_date", "status"
)

In [0]:
display(df_trace_final)